# ST590 - Project 2

Authored by Group 13 - Dave Bergeron, Xavier Genelin, and Maksim Nikiforov

## Part 2

In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import sqlite3

In [2]:
con = sqlite3.connect("northwind.sqlite")
pd.read_sql('SELECT * FROM sqlite_schema WHERE type = \'table\'', con)

,type,name,tbl_name,rootpage,sql
0,table,Categories,Categories,2,CREATE TABLE [Categories]\n( [CategoryID]...
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,CustomerCustomerDemo,CustomerCustomerDemo,28,CREATE TABLE [CustomerCustomerDemo](\n [Cust...
3,table,CustomerDemographics,CustomerDemographics,30,CREATE TABLE [CustomerDemographics](\n [Cust...
4,table,Customers,Customers,32,CREATE TABLE [Customers]\n( [CustomerID] ...
5,table,Employees,Employees,38,CREATE TABLE [Employees]\n( [EmployeeID] ...
6,table,EmployeeTerritories,EmployeeTerritories,68,CREATE TABLE [EmployeeTerritories](\n [Emplo...
7,table,Order Details,Order Details,70,CREATE TABLE [Order Details](\n [OrderID]INT...
8,table,Orders,Orders,92,CREATE TABLE [Orders](\n [OrderID]INTEGER NO...
9,table,Products,Products,128,CREATE TABLE [Products](\n [ProductID]INTEGE...


The _Employees_ table contains information on nine individuals whose employee identifications range of 1 to 9. One employee is a vice president, one employee is a sales manager, and six employees are sales representatives. The six representatives have access to one insides sales coordinator who presumably helps fulfill orders, ensures timely delivery, and assists with other logistics.   

In [3]:
pd.read_sql('SELECT EmployeeID, LastName, FirstName, Title FROM Employees', con)

,EmployeeID,LastName,FirstName,Title
0,1,Davolio,Nancy,Sales Representative
1,2,Fuller,Andrew,"Vice President, Sales"
2,3,Leverling,Janet,Sales Representative
3,4,Peacock,Margaret,Sales Representative
4,5,Buchanan,Steven,Sales Manager
5,6,Suyama,Michael,Sales Representative
6,7,King,Robert,Sales Representative
7,8,Callahan,Laura,Inside Sales Coordinator
8,9,Dodsworth,Anne,Sales Representative


We can assess the performance of each individual by understanding how much they sell. To do this, we can combine information from the _Orders_ table with additional details from the _Order Details_ table.  

In [4]:
# Merge "Orders" and "Order Details" tables to reveal the quantitites sold by each employee
# as well as the price and discount per unit
orders_per_employee = pd.merge(
    left = pd.read_sql("SELECT EmployeeID, OrderID FROM Orders", con), 
    right = pd.read_sql("SELECT OrderID,  UnitPrice, Quantity, Discount FROM [Order Details]", con),
    how = "left", 
    on = "OrderID")

# Create a new column to calculate total sales per order, inclusive of any discounts
orders_per_employee["TotalSale"] = \
(orders_per_employee["UnitPrice"]-orders_per_employee["UnitPrice"]*orders_per_employee["Discount"])*orders_per_employee["Quantity"]

# Group sales by EmployeeID and sum up total sales for each employee.
# Reset the index to allow the EmployeeID column to be merged with other tables. 
summed_sales = orders_per_employee[["EmployeeID", "Quantity", "TotalSale"]].groupby("EmployeeID").sum("TotalSale").reset_index()
summed_sales

,EmployeeID,Quantity,TotalSale
0,1,7812,192107.6045
1,2,6055,166537.7550
2,3,7852,202812.8430
3,4,9798,232890.8460
4,5,3036,68792.2825
5,6,3527,73913.1295
6,7,4654,124568.2350
7,8,5913,126862.2775
8,9,2670,77308.0665


By associating each employee ID with employee names, we see that Margaret Peacock sold the most amount of product, bringing in $\$232,890.85$ in sales (assuming a US Dollar currency). As sales representatives, Michael Suyama and Anne Dodsworth sold three times less, suggesting that there is room for improvement. 

In [13]:
# Merge EmployeeID with names and titles for better perspective
named_employee_sales = pd.merge(
    left = pd.read_sql("SELECT EmployeeID,  LastName, FirstName, Title FROM Employees", con),
    right = summed_sales, 
    how = "left", 
    on = "EmployeeID")

# Sort in descending order of total sales
named_employee_sales.sort_values(by=["TotalSale"], ascending=False)

,EmployeeID,LastName,FirstName,Title,Quantity,TotalSale
3,4,Peacock,Margaret,Sales Representative,9798,232890.8460
2,3,Leverling,Janet,Sales Representative,7852,202812.8430
0,1,Davolio,Nancy,Sales Representative,7812,192107.6045
1,2,Fuller,Andrew,"Vice President, Sales",6055,166537.7550
7,8,Callahan,Laura,Inside Sales Coordinator,5913,126862.2775
6,7,King,Robert,Sales Representative,4654,124568.2350
8,9,Dodsworth,Anne,Sales Representative,2670,77308.0665
5,6,Suyama,Michael,Sales Representative,3527,73913.1295
4,5,Buchanan,Steven,Sales Manager,3036,68792.2825


In [16]:
# Merge employee information with their associated territory ID
territory_ID = pd.merge(
    left = pd.read_sql("SELECT EmployeeID, LastName, FirstName,  TITLE FROM Employees", con), 
    right = pd.read_sql("SELECT EmployeeID,  TerritoryID FROM EmployeeTerritories", con),
    how = "inner", 
    on = "EmployeeID")

In [17]:
# Retrieve territory descriptions and region ID for each employee
territory_desc = pd.merge(
    left = territory_ID, 
    right = pd.read_sql("SELECT * FROM Territories", con),
    how = "inner", 
    on = "TerritoryID")
territory_desc

,EmployeeID,LastName,FirstName,Title,TerritoryID,TerritoryDescription,RegionID
0,1,Davolio,Nancy,Sales Representative,06897,Wilton ...,1
1,1,Davolio,Nancy,Sales Representative,19713,Neward ...,1
2,2,Fuller,Andrew,"Vice President, Sales",01581,Westboro ...,1
3,2,Fuller,Andrew,"Vice President, Sales",01730,Bedford ...,1
4,2,Fuller,Andrew,"Vice President, Sales",01833,Georgetow ...,1
5,2,Fuller,Andrew,"Vice President, Sales",02116,Boston ...,1
6,2,Fuller,Andrew,"Vice President, Sales",02139,Cambridge ...,1
7,2,Fuller,Andrew,"Vice President, Sales",02184,Braintree ...,1
8,2,Fuller,Andrew,"Vice President, Sales",40222,Louisville ...,1
9,3,Leverling,Janet,Sales Representative,30346,Atlanta ...,4


In [19]:
# Retrieve order IDs for each employee
region_desc = pd.merge(
    left = territory_desc, 
    right = pd.read_sql("SELECT * FROM Regions", con),
    how = "inner", 
    on = "RegionID")
region_desc

,EmployeeID,LastName,FirstName,Title,TerritoryID,TerritoryDescription,RegionID,RegionDescription
0,1,Davolio,Nancy,Sales Representative,06897,Wilton ...,1,Eastern ...
1,1,Davolio,Nancy,Sales Representative,19713,Neward ...,1,Eastern ...
2,2,Fuller,Andrew,"Vice President, Sales",01581,Westboro ...,1,Eastern ...
3,2,Fuller,Andrew,"Vice President, Sales",01730,Bedford ...,1,Eastern ...
4,2,Fuller,Andrew,"Vice President, Sales",01833,Georgetow ...,1,Eastern ...
5,2,Fuller,Andrew,"Vice President, Sales",02116,Boston ...,1,Eastern ...
6,2,Fuller,Andrew,"Vice President, Sales",02139,Cambridge ...,1,Eastern ...
7,2,Fuller,Andrew,"Vice President, Sales",02184,Braintree ...,1,Eastern ...
8,2,Fuller,Andrew,"Vice President, Sales",40222,Louisville ...,1,Eastern ...
9,4,Peacock,Margaret,Sales Representative,20852,Rockville ...,1,Eastern ...


In [66]:
# Retrieve sales numbers for each employee
employee_orderID = pd.merge(
    left = pd.read_sql("SELECT EmployeeID, LastName, FirstName,  TITLE FROM Employees", con), 
    right = pd.read_sql("SELECT OrderID, EmployeeID FROM Orders", con),
    how = "left", 
    on = "EmployeeID")
employee_orderID

,EmployeeID,LastName,FirstName,Title,OrderID
0,1,Davolio,Nancy,Sales Representative,10258
1,1,Davolio,Nancy,Sales Representative,10270
2,1,Davolio,Nancy,Sales Representative,10275
3,1,Davolio,Nancy,Sales Representative,10285
4,1,Davolio,Nancy,Sales Representative,10292
...,...,...,...,...,...
825,9,Dodsworth,Anne,Sales Representative,10978
826,9,Dodsworth,Anne,Sales Representative,11016
827,9,Dodsworth,Anne,Sales Representative,11017
828,9,Dodsworth,Anne,Sales Representative,11022


In [44]:
employee_orderID = pd.merge(
    left = employee_orderID, 
    right = pd.read_sql('SELECT * FROM [Order Details]', con),
    how = "left", 
    on = "OrderID")
employee_orderID

,EmployeeID,LastName,FirstName,Title,OrderID,ProductID,UnitPrice,Quantity,Discount
0,1,Davolio,Nancy,Sales Representative,10258,2,15.2,50,0.2
1,1,Davolio,Nancy,Sales Representative,10258,5,17.0,65,0.2
2,1,Davolio,Nancy,Sales Representative,10258,32,25.6,6,0.2
3,1,Davolio,Nancy,Sales Representative,10270,36,15.2,30,0.0
4,1,Davolio,Nancy,Sales Representative,10270,43,36.8,25,0.0
...,...,...,...,...,...,...,...,...,...
2150,9,Dodsworth,Anne,Sales Representative,11022,19,9.2,35,0.0
2151,9,Dodsworth,Anne,Sales Representative,11022,69,36.0,30,0.0
2152,9,Dodsworth,Anne,Sales Representative,11058,21,10.0,3,0.0
2153,9,Dodsworth,Anne,Sales Representative,11058,60,34.0,21,0.0


In [63]:
# Create new column ("TotalSale") with total price of goods for each order, taking
# into account discounts
employee_orderID["TotalSale"] = \
(employee_orderID["UnitPrice"]-employee_orderID["UnitPrice"]*employee_orderID["Discount"])*employee_orderID["Quantity"]
employee_orderID

,EmployeeID,LastName,FirstName,Title,OrderID,ProductID,UnitPrice,Quantity,Discount,TotalPrice,TotalSale
0,1,Davolio,Nancy,Sales Representative,10258,2,15.2,50,0.2,608.00,608.00
1,1,Davolio,Nancy,Sales Representative,10258,5,17.0,65,0.2,884.00,884.00
2,1,Davolio,Nancy,Sales Representative,10258,32,25.6,6,0.2,122.88,122.88
3,1,Davolio,Nancy,Sales Representative,10270,36,15.2,30,0.0,456.00,456.00
4,1,Davolio,Nancy,Sales Representative,10270,43,36.8,25,0.0,920.00,920.00
...,...,...,...,...,...,...,...,...,...,...,...
2150,9,Dodsworth,Anne,Sales Representative,11022,19,9.2,35,0.0,322.00,322.00
2151,9,Dodsworth,Anne,Sales Representative,11022,69,36.0,30,0.0,1080.00,1080.00
2152,9,Dodsworth,Anne,Sales Representative,11058,21,10.0,3,0.0,30.00,30.00
2153,9,Dodsworth,Anne,Sales Representative,11058,60,34.0,21,0.0,714.00,714.00


In [65]:
# Calculate sum of sales for each employee
employee_orderID[["EmployeeID", "Quantity", "TotalSale"]].groupby("EmployeeID").sum("TotalPrice")

,Quantity,TotalSale
EmployeeID,,
1,7812,192107.6045
2,6055,166537.7550
3,7852,202812.8430
4,9798,232890.8460
5,3036,68792.2825
6,3527,73913.1295
7,4654,124568.2350
8,5913,126862.2775
9,2670,77308.0665


In [34]:
pd.read_sql('SELECT * FROM Orders', con)

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,1996-07-04 00:00:00.000,1996-08-01 00:00:00.000,1996-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l-Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05 00:00:00.000,1996-08-16 00:00:00.000,1996-07-10 00:00:00.000,1,11.61,Toms SpezialitÃ¤ten,Luisenstr. 48,MÃ¼nster,None,44087,Germany
2,10250,HANAR,4,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-12 00:00:00.000,2,65.83,Hanari Carnes,"Rua do PaÃ§o, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08 00:00:00.000,1996-08-05 00:00:00.000,1996-07-15 00:00:00.000,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09 00:00:00.000,1996-08-06 00:00:00.000,1996-07-11 00:00:00.000,2,51.30,SuprÃªmes dÃ©lices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05 00:00:00.000,1998-06-02 00:00:00.000,None,2,24.95,Pericles Comidas clÃ¡sicas,Calle Dr. Jorge Cash 321,MÃ©xico D.F.,None,5033,Mexico
826,11074,SIMOB,7,1998-05-06 00:00:00.000,1998-06-03 00:00:00.000,None,2,18.44,Simons bistro,VinbÃ¦ltet 34,Kobenhavn,None,1734,Denmark
827,11075,RICSU,8,1998-05-06 00:00:00.000,1998-06-03 00:00:00.000,None,2,6.19,Richter Supermarkt,Starenweg 5,GenÃ¨ve,None,1204,Switzerland
828,11076,BONAP,4,1998-05-06 00:00:00.000,1998-06-03 00:00:00.000,None,2,38.28,Bon app-,"12, rue des Bouchers",Marseille,None,13008,France


In [32]:
pd.read_sql('SELECT * FROM [Order Details]', con)

,OrderID,ProductID,UnitPrice,Quantity,Discount
0,10248,11,14.00,12,0.00
1,10248,42,9.80,10,0.00
2,10248,72,34.80,5,0.00
3,10249,14,18.60,9,0.00
4,10249,51,42.40,40,0.00
...,...,...,...,...,...
2150,11077,64,33.25,2,0.03
2151,11077,66,17.00,1,0.00
2152,11077,73,15.00,2,0.01
2153,11077,75,7.75,4,0.00
